# Скрипт по сбору данных

Данный скрипт - шаблонный для нашей группы.

Мы используем конфигурационные файлы, которые удобно редактировать. почти все библиотеки встроенные.

Файлы проекта (required):

1) .ipynb (парсинг каналов по тематикам и исполнителям - шаблон)
4) config.yml (YAML файл с конфигурацией для создания клиента и создания запроса)
5) Сведенный датасет


## Инициализация скрипта

In [1]:
import pathlib

#1
# Loading Libraries

import pandas as pd
import googleapiclient.discovery as api
import numpy as np
import requests

from tqdm import tqdm

from pathlib import Path
import os
import yaml
import json

In [2]:
#2

# Defining variables

# Determine path to the working directory
path = Path().resolve()
print('Current Path')
print(path)

# Opening and parsing yaml file
if path:

    cnf_path = str(path) + '/' + 'config.yml'

    with open(cnf_path, 'r') as f:
        config = yaml.safe_load(f)

    print('Loaded with Pathlib ')
else:
    try:
        with open('config.yml', 'r') as f:
            config = yaml.safe_load(f)
    except:
        print('Попробуете распаковать проект целиком?')

DATA_PATH = pathlib.Path().resolve() / config['api']['DATA_PATH']

if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)
print('ПУТЬ ДАННЫХ:', DATA_PATH)

Current Path
/Users/andrey/PycharmProjects/DataMiningProjects/notebooks/HW1
Loaded with Pathlib 
ПУТЬ ДАННЫХ: /Users/andrey/PycharmProjects/DataMiningProjects/notebooks/HW1/data


# Создание клиента запроса

In [3]:
# 6 Определение API-Key Storage

# 3.1
class KeyStorage:
    """
    Class to use in iteration
    """

    def __init__(self, key_arr: list[str]):
        self.key_gen = iter(key_arr)

    def get_next_key(self):
        """
        Returns API key to use
        :return:
        """
        return next(self.key_gen)

key_storage = KeyStorage(config['api']['keys'])

# Запрос: поиск бьюти блоггеров
## Парсинг данных из Youtube


In [4]:
#6

# Создаем параметры нашего поиска

query_params = {
    "type": "channel",
    "maxResults": 50,
    "part": "snippet",
    "regionCode": "RU",
}

display(query_params)

{'type': 'channel', 'maxResults': 50, 'part': 'snippet', 'regionCode': 'RU'}

In [7]:
# 7 Создание клиента
api_name = "youtube"
api_version = "v3"
cur_key = key_storage.get_next_key()

def get_api(cur_key):
    api_client = api.build(
            api_name,
            api_version,
            developerKey=cur_key
        )
    return api_client


youtube = get_api(cur_key)

query = 'корейская косметика'


In [8]:
# 8 Поиск без order

request = youtube.search().list(
        q = query,
        **query_params
)
response = request.execute()

df_supplemented = pd.json_normalize(response['items'])

    # Цикл для прохода по всем следующим страницам с выдачей:
i = 0
while 'nextPageToken' in response.keys():
    request = youtube.search().list(
            q = query,
            pageToken = response['nextPageToken'],
            **query_params
        )
    response = request.execute()

    print(f'Итерация БЕЗ включения аргумента order №{i}')
    i += 1

    df_additional = pd.json_normalize(response['items'])
    df_supplemented = pd.concat([df_supplemented, df_additional])



print(f"Искомых объектов {response['pageInfo']['totalResults']}", \
      f"а найденных БЕЗ включения каких-либо значений аргумента order {len(df_supplemented.drop_duplicates('id.channelId'))}")


print('Размер получившегося датафрейма', df_supplemented.shape)

Итерация БЕЗ включения аргумента order №0
Итерация БЕЗ включения аргумента order №1
Итерация БЕЗ включения аргумента order №2
Итерация БЕЗ включения аргумента order №3
Итерация БЕЗ включения аргумента order №4
Итерация БЕЗ включения аргумента order №5
Итерация БЕЗ включения аргумента order №6
Итерация БЕЗ включения аргумента order №7
Итерация БЕЗ включения аргумента order №8
Итерация БЕЗ включения аргумента order №9
Искомых объектов 6265 а найденных БЕЗ включения каких-либо значений аргумента order 470
Размер получившегося датафрейма (509, 14)


In [9]:
# 9 Варьирование значения аргумента order:
order_list = ['date', 'rating', 'title', 'videoCount', 'viewCount']

In [10]:
# 10
# Цикл для прохода по значениям аргумента order, внутри которых проход по всем страницам выдачи:
i = 0
for order in order_list:
    # Для остановки алгоритма, если все искомые объекты найдены
    # БЕЗ включения каких-либо значений аргумента order (в т.ч. вообще БЕЗ них):
    if len(df_supplemented.drop_duplicates('id.channelId')) < response['pageInfo']['totalResults']:

        # Первый заход с каждым значение аргумента order:
        request = youtube.search().list(q = query,
                                        order=order,
                                        **query_params
        )
        response = request.execute()

        # Визуализация процесса:
        print(f'Итерация №{i}, "order" {order}, "items" {len(response["items"])}')
        i += 1

        # Занесение собранных данных в итоговую таблицу df_supplemented:
        df = pd.json_normalize(response["items"])
        df_supplemented = pd.concat([df, df_supplemented])

        # Заходы с тем же значением аргумента order на следующие страницы:
        while ('nextPageToken' in response.keys())\
        & (len(df_supplemented.drop_duplicates('id.channelId')) < response['pageInfo']['totalResults'])\
        & (len(response["items"]) > 0):
        # второе условие -- для остановки алгоритма, если все искомые объекты найдены
        # БЕЗ какой-то из следующих страниц ( в т.ч. вообще БЕЗ них)
        # третье условие -- для остановки алгоритма, если предыдущая страница выдачи содержит 0 объектов

            request = youtube.search().list(
                                            q = query,
                                            pageToken = response['nextPageToken'],
                                            **query_params
            )
            response = request.execute()

            # Визуализация процесса:
            print(f'Итерация №{i}, "order" {order}, "items" {len(response["items"])}')

            i += 1

            # Занесение собранных данных в итоговую таблицу df_supplemented:
            df_additional = pd.json_normalize(response["items"])
            df_supplemented = pd.concat([df_supplemented, df_additional])

        # Сохранение составленных для каждого значения аргумента order таблиц в Excel -- на всякий случай:
        df_supplemented.to_excel(DATA_PATH / f'Каналы_{query}_Sorted_by_{order}.xlsx')
    else:
        print('Все искомые объекты найдены БЕЗ включения некоторых значений аргумента order (в т.ч. вообще БЕЗ них)')

Итерация №0, "order" date, "items" 50
Итерация №1, "order" date, "items" 50
Итерация №2, "order" date, "items" 50
Итерация №3, "order" date, "items" 50
Итерация №4, "order" date, "items" 50
Итерация №5, "order" date, "items" 50
Итерация №6, "order" date, "items" 50
Итерация №7, "order" date, "items" 50
Итерация №8, "order" date, "items" 50
Итерация №9, "order" date, "items" 50
Итерация №10, "order" date, "items" 8
Итерация №11, "order" rating, "items" 50
Итерация №12, "order" rating, "items" 50
Итерация №13, "order" rating, "items" 50
Итерация №14, "order" rating, "items" 50
Итерация №15, "order" rating, "items" 50
Итерация №16, "order" rating, "items" 50
Итерация №17, "order" rating, "items" 50
Итерация №18, "order" rating, "items" 50
Итерация №19, "order" rating, "items" 50
Итерация №20, "order" rating, "items" 50
Итерация №21, "order" rating, "items" 9
Итерация №22, "order" title, "items" 50
Итерация №23, "order" title, "items" 50
Итерация №24, "order" title, "items" 50
Итерация №25

# Просмотр полученного DataFrame и его преобразование

In [11]:
# 11

df_search = df_supplemented.copy(deep=True)
df_search = df_search.drop_duplicates(subset=['id.channelId'],)

df_search.head()


,kind,etag,id.kind,id.channelId,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.medium.url,snippet.thumbnails.high.url,snippet.channelTitle,snippet.liveBroadcastContent,snippet.publishTime
0,youtube#searchResult,KD4wB0kFCleorLAZUqunFeeNnGU,youtube#channel,UCyBkYEimyewSS9Tf_XasxqA,2019-09-26T16:51:01Z,UCyBkYEimyewSS9Tf_XasxqA,Chok - Chok Корейская косметика,1.,https://yt3.ggpht.com/ytc/AL5GRJXSaaDeBjL92F2Y...,https://yt3.ggpht.com/ytc/AL5GRJXSaaDeBjL92F2Y...,https://yt3.ggpht.com/ytc/AL5GRJXSaaDeBjL92F2Y...,Chok - Chok Корейская косметика,none,2019-09-26T16:51:01Z
1,youtube#searchResult,rYW08nw-AIqP6rHhabVqo3r4GOo,youtube#channel,UCPmSyu4_Xxw2d8uOThJ5tCQ,2015-04-01T11:37:55Z,UCPmSyu4_Xxw2d8uOThJ5tCQ,Корейская косметика,,https://yt3.ggpht.com/ytc/AL5GRJX9KQ-jcjyvtAC8...,https://yt3.ggpht.com/ytc/AL5GRJX9KQ-jcjyvtAC8...,https://yt3.ggpht.com/ytc/AL5GRJX9KQ-jcjyvtAC8...,Корейская косметика,none,2015-04-01T11:37:55Z
2,youtube#searchResult,hq-V8AVxdQs4HTDfyp9T7YaCWak,youtube#channel,UCIH9WX73cmksrcE4lKzns2w,2021-01-11T04:37:21Z,UCIH9WX73cmksrcE4lKzns2w,Корейская люкс косметика,Корейская люкс косметика Здоровая кожа - залог...,https://yt3.ggpht.com/HE9yWbxQN3MDaXFmyJeytKHc...,https://yt3.ggpht.com/HE9yWbxQN3MDaXFmyJeytKHc...,https://yt3.ggpht.com/HE9yWbxQN3MDaXFmyJeytKHc...,Корейская люкс косметика,none,2021-01-11T04:37:21Z
3,youtube#searchResult,zpmgla_nnagIcn9rP_2JLgLaiYo,youtube#channel,UCmMHjFuHh8lseNR114GP_ZQ,2021-07-02T19:14:44Z,UCmMHjFuHh8lseNR114GP_ZQ,EandE Cosmetics,EandE Cosmetics. Волоколамск ⭐️Наш первый Шоу-...,https://yt3.ggpht.com/ArUV5fa3-ey0FC-X11THKv-r...,https://yt3.ggpht.com/ArUV5fa3-ey0FC-X11THKv-r...,https://yt3.ggpht.com/ArUV5fa3-ey0FC-X11THKv-r...,EandE Cosmetics,none,2021-07-02T19:14:44Z
4,youtube#searchResult,26Gb5w9PCwul3ZBjH3XPbWR_beY,youtube#channel,UCFmgLIGI4CTB_g7LEw_VXfg,2018-05-16T08:08:08Z,UCFmgLIGI4CTB_g7LEw_VXfg,Корейская Косметика,,https://yt3.ggpht.com/ytc/AL5GRJX5-fC0j0POMv5F...,https://yt3.ggpht.com/ytc/AL5GRJX5-fC0j0POMv5F...,https://yt3.ggpht.com/ytc/AL5GRJX5-fC0j0POMv5F...,Корейская Косметика,none,2018-05-16T08:08:08Z


In [12]:
# 12

df_search.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 669 entries, 0 to 17
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   kind                            669 non-null    object
 1   etag                            669 non-null    object
 2   id.kind                         669 non-null    object
 3   id.channelId                    669 non-null    object
 4   snippet.publishedAt             669 non-null    object
 5   snippet.channelId               669 non-null    object
 6   snippet.title                   669 non-null    object
 7   snippet.description             669 non-null    object
 8   snippet.thumbnails.default.url  669 non-null    object
 9   snippet.thumbnails.medium.url   669 non-null    object
 10  snippet.thumbnails.high.url     669 non-null    object
 11  snippet.channelTitle            669 non-null    object
 12  snippet.liveBroadcastContent    669 non-null    obj

In [13]:
df_search.to_excel(DATA_PATH / 'Channels_search.xlsx', index=False)

In [14]:
# 13
# Загрузка характеристик видео

search_parts = [
    "brandingSettings",
    "contentDetails",
    "contentOwnerDetails",
    "snippet",
    "statistics",
    "topicDetails",
]

# список каналов
channel_ids = list(df_search['id.channelId'])

In [15]:
# 14 Множественный вызов по 50 элементов

df_supplemented = pd.DataFrame()

for i in range(0, len(df_search), 50):
    # границы интервала
    s, e = i, i+50
    #if e > len(df_search): e = len(df_search)

    request = youtube.channels().list(
    part=search_parts,
    maxResults=50,
    id=channel_ids[s:e],
    )
    response = request.execute()

    df_additional = pd.json_normalize(response['items'])
    df_supplemented = pd.concat([df_supplemented, df_additional])
    print(f"Итерация {i // 50} завершена")#

Итерация 0 завершена
Итерация 1 завершена
Итерация 2 завершена
Итерация 3 завершена
Итерация 4 завершена
Итерация 5 завершена
Итерация 6 завершена
Итерация 7 завершена
Итерация 8 завершена
Итерация 9 завершена
Итерация 10 завершена
Итерация 11 завершена
Итерация 12 завершена
Итерация 13 завершена


In [16]:
# 15 Валидация количества каналов

if len(df_search) == len(df_supplemented): print('Всё ок')
else: print("Не досчитались:", abs(len(df_search) - len(df_supplemented)), "каналов")

Всё ок


In [17]:
# 16
df_channel = df_supplemented.copy(deep=True)
df_channel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 669 entries, 0 to 18
Data columns (total 36 columns):
 #   Column                                               Non-Null Count  Dtype 
---  ------                                               --------------  ----- 
 0   kind                                                 669 non-null    object
 1   etag                                                 669 non-null    object
 2   id                                                   669 non-null    object
 3   snippet.title                                        669 non-null    object
 4   snippet.description                                  669 non-null    object
 5   snippet.customUrl                                    669 non-null    object
 6   snippet.publishedAt                                  669 non-null    object
 7   snippet.thumbnails.default.url                       669 non-null    object
 8   snippet.thumbnails.default.width                     669 non-null    int64 
 9   

In [34]:
# 17 Преобразование числовых колонок для корректного отображения

numeric_columns = [
    "contentDetails.relatedPlaylists.likes",
    "contentDetails.relatedPlaylists.uploads",
    'statistics.viewCount',
       'statistics.subscriberCount', 'statistics.hiddenSubscriberCount',
       'statistics.videoCount'
]



for num_col in numeric_columns:

    df_channel[num_col] = pd.to_numeric(df_channel[num_col], errors='coerce')


In [35]:
# 18 Отображение числовых характеристик для фильтрации
pd.set_option('float_format', '{:f}'.format)

df_channel[numeric_columns].describe()

,contentDetails.relatedPlaylists.likes,contentDetails.relatedPlaylists.uploads,statistics.viewCount,statistics.subscriberCount,statistics.videoCount
count,0.000000,0.000000,669.000000,669.000000,669.000000
mean,NaN,NaN,903640.536622,6358.892377,36.451420
std,NaN,NaN,17795462.588440,118337.944930,95.796095
min,NaN,NaN,0.000000,0.000000,0.000000
25%,NaN,NaN,143.000000,2.000000,3.000000
50%,NaN,NaN,1072.000000,11.000000,10.000000
75%,NaN,NaN,9231.000000,52.000000,26.000000
max,NaN,NaN,458127984.000000,3040000.000000,1213.000000


# Фильтрация




In [30]:
# 19 Вывод характеристик для фильтрации

view_filter = np.percentile(df_channel['statistics.viewCount'], 95)
sub_filter = np.percentile(df_channel['statistics.subscriberCount'], 95)

print('Отсев по просмотрам', view_filter)
print('Отсев по подписчикам', sub_filter)

Отсев по просмотрам 451379.2000000006
Отсев по подписчикам 3698.0000000000223


1. Количество просмотров - 95% (медиана)
2. Количество видео - 95% (медиана)

In [31]:
# 20 Статистика после фильтра
df_final_channels = df_channel[(df_channel['statistics.viewCount'] >= view_filter) | (df_channel['statistics.subscriberCount'] >= sub_filter)]

df_final_channels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 43 to 14
Data columns (total 36 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   kind                                                 40 non-null     object 
 1   etag                                                 40 non-null     object 
 2   id                                                   40 non-null     object 
 3   snippet.title                                        40 non-null     object 
 4   snippet.description                                  40 non-null     object 
 5   snippet.customUrl                                    40 non-null     object 
 6   snippet.publishedAt                                  40 non-null     object 
 7   snippet.thumbnails.default.url                       40 non-null     object 
 8   snippet.thumbnails.default.width                     40 non-null     in

In [32]:
# 21 Сохранение

df_final_channels.to_excel(DATA_PATH / f'{query}_channels.xlsx', index=False)

# Общее сведение 2-х датасетов

In [36]:
query2 = 'бьюти блогеры'

df1 = pd.read_excel(DATA_PATH / f'{query}_channels.xlsx')
df2= pd.read_excel(DATA_PATH / f'{query2}_channels.xlsx')

df = pd.concat([df1, df2])

df = df.drop_duplicates(subset=['id'])

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 205 entries, 0 to 168
Data columns (total 36 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   kind                                                 205 non-null    object 
 1   etag                                                 205 non-null    object 
 2   id                                                   205 non-null    object 
 3   snippet.title                                        205 non-null    object 
 4   snippet.description                                  194 non-null    object 
 5   snippet.customUrl                                    205 non-null    object 
 6   snippet.publishedAt                                  205 non-null    object 
 7   snippet.thumbnails.default.url                       205 non-null    object 
 8   snippet.thumbnails.default.width                     205 non-null    i

Общий массив для работы - 205 каналов

In [37]:
df.to_excel(DATA_PATH / 'channels_union_base.xlsx', index=False)